<a href="https://colab.research.google.com/github/Aksh5/Capstone/blob/main/EDA_Folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install folium
!pip3 install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=c6370945b664e066d10755d5ca2f815a9748ac697633e84daa146c9ba25aa516
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [2]:
import folium
import wget
import pandas as pd

In [3]:
from folium.plugins import MarkerCluster


In [4]:
from folium.plugins import MousePosition

from folium.features import DivIcon


In [5]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [6]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


In [7]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

In [8]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))


In [9]:
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [10]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
sites_list = launch_sites_df['Launch Site'].values.tolist()
coordinate_list = launch_sites_df[['Lat', 'Long']].values.tolist()

In [11]:
i = 0
while i < len(sites_list):
    circle = folium.Circle(coordinate_list[i], radius=100, color='#d35400', fill=True).add_child(folium.Popup(str(sites_list[i])))
    marker = folium.map.Marker(
        coordinate_list[i],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % str(sites_list[i]),
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)
    i = i+1
# launch_sites_df
site_map

In [12]:
spacex_df.tail(10)


,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [13]:
marker_cluster = MarkerCluster()


In [14]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [19]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


In [21]:
dict = {'Launch Site': ['CCAFS LC-40','CCAFS SLC-40','KSC LC-39A','VAFB SLC-4E'], 
        'coastline_lat': [28.56256,28.56413,28.58674,34.63626],
        'coastline_long': [-80.56784,-80.56811,-80.66231,-120.6249],
        'rail_lat': [28.57205,28.57205,28.57282,34.63394],
        'rail_long': [-80.58525,-80.58525,-80.6539,-120.62457]}
dict_df = pd.DataFrame.from_dict(dict)
df = launch_sites_df.join(dict_df.set_index('Launch Site'), on='Launch Site')
j = 0
while j < len(sites_list):
    distance = calculate_distance(df['Lat'][j], df['Long'][j], df['coastline_lat'][j], df['coastline_long'][j])
    print('Distance from ' + str(df['Launch Site'][j]) + ' to nearest coastline is: %f km' % distance )
    j = j+1

Distance from CCAFS LC-40 to nearest coastline is: 0.930134 km
Distance from CCAFS SLC-40 to nearest coastline is: 0.857200 km
Distance from KSC LC-39A to nearest coastline is: 2.125310 km
Distance from VAFB SLC-4E to nearest coastline is: 1.350289 km


In [22]:
k = 0
while k <len(sites_list):
    distance = calculate_distance(df['Lat'][k], df['Long'][k], df['coastline_lat'][k], df['coastline_long'][k])
    distance_marker = folium.Marker(
       [df['coastline_lat'][k],df['coastline_long'][k]],
       icon=DivIcon(
           icon_size=(20,20),
           icon_anchor=(0,0),
           html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
           )
       )
    site_map.add_child(distance_marker)
    k = k+1
site_map

In [23]:
k = 0
while k <len(sites_list):
    lines=folium.PolyLine(locations=[[df['Lat'][k], df['Long'][k]], [df['coastline_lat'][k], df['coastline_long'][k]]], weight=1)
    site_map.add_child(lines)
    k = k+1
site_map

In [24]:
k = 0
while k <len(sites_list):
    distance = calculate_distance(df['Lat'][k], df['Long'][k], df['rail_lat'][k], df['rail_long'][k])
    distance_marker = folium.Marker(
       [df['rail_lat'][k],df['rail_long'][k]],
       icon=DivIcon(
           icon_size=(20,20),
           icon_anchor=(0,0),
           html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
           )
       )
    
    lines=folium.PolyLine(locations=[[df['Lat'][k], df['Long'][k]], [df['rail_lat'][k], df['rail_long'][k]]], weight=1)
    site_map.add_child(distance_marker)
    site_map.add_child(lines)
    k = k+1
site_map